#  ECB cut-and-paste

Write a k=v parsing routine, as if for a structured cookie. The routine should take:

foo=bar&baz=qux&zap=zazzle

... and produce:

{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}


In [2]:
import sys

print(sys.path)

['/home/c4ndyp0p/sem6/matasano-cryptopals/set2', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/c4ndyp0p/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [3]:
from aes import *

In [4]:
string = "foo=bar&baz=qux&zap=zazzle"

outString = {}

for i in string.split("&"):
    print(i)
    outString[i.split("=")[0]] = i.split("=")[1]

print(outString)


foo=bar
baz=qux
zap=zazzle
{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}


Now write a function that encodes a user profile in that format, given an email address. You should have something like:

profile_for("foo@bar.com")
... and it should produce:

{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
... encoded as:

email=foo@bar.com&uid=10&role=user

In [5]:
def profile_for(email):
    email = email.replace("&", "")
    email = email.replace("=", "")
    return "email=" + email + "&uid=10&role=user"

In [6]:
user_profile = profile_for("foo.bar")

print(user_profile)

email=foo.bar&uid=10&role=user


Your "profile_for" function should not allow encoding metacharacters (& and =). Eat them, quote them, whatever you want to do, but don't let people set their email address to "foo@bar.com&role=admin".

Now, two more easy functions. Generate a random AES key, then:

Encrypt the encoded user profile under the key; "provide" that to the "attacker".
Decrypt the encoded user profile and parse it.
Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile

In [7]:
# Generate a random AES key, then:

# Encrypt the encoded user profile under the key; "provide" that to the "attacker".

key = b'dTP\xdc"\xb8\xa4\x12\xe3H\xfc\xd5zS\x02\xf2'
user_profile_bytes = PKCS7_padding(user_profile.encode(), 16)
print(user_profile_bytes)

encrypted_profile = encrypt_aes_ecb(user_profile_bytes, key)

print(encrypted_profile)



b'email=foo.bar&uid=10&role=user\x02\x02'
b'Ys\xa0\xbb\x83\xa6\xce.\xa3\xa3\xd9\xb4E!\xa4t\x9c\x1f\x00\x8a\xd0_\x04-\x00\x88.\xaf\x85\x1f\xd7\x97'


In [8]:
# Decrypt the encoded user profile and parse it.

decrypted_profile = decrypt_aes_ecb(encrypted_profile, key)
print(decrypted_profile)



# Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile

b'email=foo.bar&uid=10&role=user\x02\x02'


In [12]:
# 1. get ct of "email=foo@bar12." "com&uid=10&role=" "user____________"
# save ct 1 and ct 2

final_ct = []

profile_1 = profile_for("foo@bar12.com")
profile_1_bytes = PKCS7_padding(profile_1.encode(), 16)
print(profile_1_bytes)

encrypted_profile_1 = encrypt_aes_ecb(profile_1_bytes, key)

print(encrypted_profile_1)

ct_1 = get_blocks_in_list(encrypted_profile_1, 16)
final_ct.append(ct_1[0])
final_ct.append(ct_1[1])

print(final_ct)

b'email=foo@bar12.com&uid=10&role=user\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c\x0c'
b'\xae\x01}\xd2M\x94\\\xf0\xde\xc5\xb9\x83\x86\xd4K\x1b\xcf\xdb\x8eT;-C\xceJ\xb7\x98\x19P\x12}\xe3,\xf7<Y\xcf\xec\xe4\x93\x03+\xab%\xfbN\x12\xaf'
[b'\xae\x01}\xd2M\x94\\\xf0\xde\xc5\xb9\x83\x86\xd4K\x1b', b'\xcf\xdb\x8eT;-C\xceJ\xb7\x98\x19P\x12}\xe3']


In [19]:
# get ct of "admin____________"

payload1 = PKCS7_padding("admin".encode(), 16)
print(payload1)

profile_2 = profile_for("foo@bar12.admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b")
profile_2_bytes = PKCS7_padding(profile_2.encode(), 16)
print(get_blocks_in_list(profile_2_bytes, 16))

encrypted_profile_2 = encrypt_aes_ecb(profile_2_bytes, key)
final_ct.append(get_blocks_in_list(encrypted_profile_2, 16)[1])

b'admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'
[b'email=foo@bar12.', b'admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b', b'&uid=10&role=use', b'r\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f']


In [20]:
print(final_ct)

[b'\xae\x01}\xd2M\x94\\\xf0\xde\xc5\xb9\x83\x86\xd4K\x1b', b'\xcf\xdb\x8eT;-C\xceJ\xb7\x98\x19P\x12}\xe3', b"\xdd\xdb\x15\x80\x88\xfdY\x03aP'\xad\xfc\xcd\x0e\xd8"]


In [22]:
#verify

final_ct_check = bytearray(final_ct[0] + final_ct[1] + final_ct[2])

print(decrypt_aes_ecb(final_ct_check, key))

b'email=foo@bar12.com&uid=10&role=admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'
